In [113]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav

import numpy as np
import os
import random 
import operator
import math
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import librosa
from sklearn.model_selection import train_test_split
import pandas as pd

In [114]:
genres_dict = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
df = pd.DataFrame(columns=['music_vectorized', 'genre'])

# Convert .wav files into Feature Vectors

In [115]:
vectors = []
genres = []
#first load song as floating point time series, where we create 2 second snippets for data (we could make this smaller for more data)
# y = audio time series
# sr = sampling rate of y
for genre, genre_numb in genres_dict.items():
    for filename in os.listdir(f'./data/genres/{genre}'):
        song_path = f'./data/genres/{genre}/{filename}'
        for i in range(14):
            y, sr = librosa.load(song_path, mono=True, duration=2, offset=i*2)
            ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512, n_mels=64) #obtain feature vectors of music
            ps = librosa.power_to_db(ps**2) #scale
            vectors.append(ps)   
            genres.append(genre_numb)

In [116]:
df['music_vectorized'] = vectors
df['genre'] = genres

In [117]:
df

,music_vectorized,genre
0,"[[0.36136138, -11.7266035, -4.977748, -5.66650...",0
1,"[[-33.736053, -18.770893, -12.80958, -24.47540...",0
2,"[[-0.82316035, 4.6949806, 2.617299, 9.172368, ...",0
3,"[[-3.8815877, -5.6365113, -13.949097, -9.71283...",0
4,"[[-8.264401, -20.482605, -18.05536, -13.360597...",0
...,...,...
13995,"[[1.0828435, 12.707238, -3.6865675, 2.7202482,...",9
13996,"[[-39.66436, 5.0170617, -7.283575, 4.192057, 3...",9
13997,"[[-9.38166, -4.1743274, -8.460716, -1.1189616,...",9
13998,"[[-1.0145975, 5.2836924, -2.4371595, -3.901622...",9


# Create training and testing data

In [118]:
X_train, X_test, y_train, y_test = train_test_split(np.array(df['music_vectorized'].to_list()), np.array(df['genre'].to_list()), test_size=0.3, random_state = 123)
nsamples, nx, ny = X_train.shape
d2_X_train = X_train.reshape((nsamples,nx*ny))

In [119]:
nsamples, nx, ny = X_test.shape
d2_X_test = X_test.reshape((nsamples, nx*ny))

# Fit Model

In [120]:
clf = KNeighborsClassifier()
clf = clf.fit(d2_X_train, y_train)
y_pred = clf.predict(d2_X_test)

In [121]:
from sklearn.metrics import accuracy_score

In [122]:
accuracy_score(y_test, y_pred)

0.4297619047619048